## Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN)
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
Grid Search/Random Search: Use grid search or random search to try different architectures.
Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [8]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.3.1

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2


You can safely remove it manually.


   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.2 MB 12.6 MB/s eta 0:00:01
   --------------------- ------------------ 5.0/9.2 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------  9.2/9.2 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 9.2/9.2 MB 15.5 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.
scikeras 0.13.0 requires scikit-learn>=1.4.2, but you have scikit-learn 1.3.1 which is incompatible.


In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [5]:
## Create a Keras classifier
model=KerasClassifier(neurons=32,layers=1,build_fn=create_model,verbose=1)

In [6]:
## define the grid search parameters
param_grid = {
    'neurons' : [16, 32, 64, 128],
    'layers' : [1, 2, 3],
    'epochs' : [50, 100]
}

In [11]:
# Perform grid search
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1, metrics = ['accuracy'])
grid_result = grid.fit(X_train, y_train)

# Best result
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

TypeError: GridSearchCV.__init__() got an unexpected keyword argument 'metrics'

In [10]:
X_train.shape

(8000, 12)

In [25]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      2.2.2
altair                       5.5.0
asttokens                    3.0.0
astunparse                   1.6.3
attrs                        25.3.0
blinker                      1.9.0
cachetools                   5.5.2
certifi                      2025.1.31
charset-normalizer           3.4.1
click                        8.1.8
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.3.1
cycler                       0.12.1
debugpy                      1.8.13
decorator                    5.2.1
executing                    2.2.0
flatbuffers                  25.2.10
fonttools                    4.57.0
gast                         0.6.0
gitdb                        4.0.12
GitPython                    3.1.44
google-auth                  2.38.0
google-auth-oauthlib         1.2.1
google-pasta                 0.2.0
grpcio                       1.71.

In [24]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.15.0

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0


You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.


  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl (300.9 MB)
